In [1]:
#Needed Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import re

import os
import sys

from tqdm import tqdm

import json
from pathlib import Path

import matplotlib.pyplot as plt

print("Starting Notebook.")

sns.set(font_scale = 1.25)
sns.set_style("whitegrid")

Starting Notebook.


In [2]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 
           'JFrog', 'Jira', 'JiraEcosystem', 
           'MariaDB', 'MongoDB', 'Qt', 
           'RedHat', 'Sakai', 'SecondLife', 
           'Sonatype', 'Spring']
#Mindville
CONFIG = ['R_LTvNL', 'R_LTvNLOL', 'R_LTOLvNL']
LT = 'Duplication'

## Loading Data

In [3]:
def print_linktypes(SOURCE):
    #Loading Issues
    filename = '../../data/crawl/issues_'+SOURCE.lower()+'.csv'
    issues = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    
    issue_set = set(issues['issue_id'])
        
    #Loading Links
    filename = '../../data/crawl/clean_links_'+SOURCE.lower()+'.csv'
    links = pd.read_csv(filename, encoding="UTF-8", low_memory=False, index_col=0, sep=';')

    link_set = set(links['issue_id_1']).union(set(links['issue_id_2']))

    num_dups = len(links[links['linktype']=='Duplicate'])
        
    return len(issues), len(links), len(links.linktype.unique()), round(len(link_set)/len(issue_set), 3), num_dups

In [4]:
overview = pd.DataFrame(columns = ['Project', '#Issues', '#Links', '#Linktypes', '%IssuesWithLinks', '#NumDups'])
j=0
for s in SOURCES:
    i, l, ltu, pi, nd = print_linktypes(s)
    
    if s == 'JiraEcosystem':
        s = 'JiraEco.'
    
    overview.loc[j]=[s, i, l, ltu, pi, nd]
    
    j+=1

In [5]:
overview.sort_values(by=['#Issues'], ascending=False)

,Project,#Issues,#Links,#Linktypes,%IssuesWithLinks,#NumDups
0,Apache,1014926,255767,22,0.285,25925
9,RedHat,353000,119669,21,0.392,5913
4,Jira,274545,99819,19,0.467,21685
8,Qt,148579,40105,12,0.302,4243
7,MongoDB,137172,63821,15,0.452,8587
12,Sonatype,87284,4465,11,0.070,342
13,Spring,69156,14462,9,0.256,1745
10,Sakai,50550,19803,8,0.424,0
5,JiraEco.,41866,11398,20,0.330,1741
6,MariaDB,31229,14618,8,0.445,1374


## Loading Model Results

In [6]:
valid_projects = []

for s in SOURCES:
    if (s == "MariaDB"):
        valid = True
        for c in CONFIG:
            filename = 'results/dccnn_'+s.lower()+'_'+LT+'_'+c+'_metrics.csv'
            metrics_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False)

            LT_mets = metrics_df.iloc[0].values.tolist()[1:]
            NL_mets = metrics_df.iloc[1].values.tolist()[1:]
            OL_mets = metrics_df.iloc[2].values.tolist()[1:]
            valid = valid and not(np.isnan(LT_mets + OL_mets + NL_mets).any())

        if valid:
            valid_projects.append(s)

In [7]:
valid_projects

['MariaDB']

In [8]:
def get_tpfptnfn(conf_mat):
    
    tp = conf_mat.loc["DUPLICATION"][1] 
    fn = conf_mat.loc["DUPLICATION"][0] 
    fp = conf_mat.loc["NON-LINKS"][1] 
    tn = conf_mat.loc["NON-LINKS"][0] 
    
    return tp, fp, tn, fn

In [9]:
def get_results(c, trad):
    avg_d_pre = []
    avg_d_rec = []
    avg_d_f1 = []

    avg_nl_pre = []
    avg_nl_rec = []
    avg_nl_f1 = []

    avg_ol_0 = []
    avg_ol_1 = []

    avg_acc = []

    avg_pre = []
    avg_rec = []
    avg_f1 = []

    for s in valid_projects:
#         print(s.upper())
        filename = 'results/dccnn_'+s.lower()+'_'+LT+'_'+c+'_confmat.csv'
        confmat_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, index_col='Class')
        confmat_df = pd.DataFrame(confmat_df.values, index=['DUPLICATION', 'OTHER-LINKS', 'NON-LINKS'])

        avg_ol_0.append((confmat_df.transpose()/confmat_df.sum(axis=1)).loc[0]["OTHER-LINKS"])
        avg_ol_1.append((confmat_df.transpose()/confmat_df.sum(axis=1)).loc[1]["OTHER-LINKS"])

#         print("OL 0: "+ str(np.round((confmat_df.transpose()/confmat_df.sum(axis=1)).loc['0']["OTHER-LINKS"], 3)))
#         print("OL 1: "+ str(np.round((confmat_df.transpose()/confmat_df.sum(axis=1)).loc['1']["OTHER-LINKS"], 3)))
        
        if not trad:
            if c == 'R_LTOLvNL':
                new_confmat = [[confmat_df.loc["DUPLICATION"][0]+confmat_df.loc["OTHER-LINKS"][0], 
                                confmat_df.loc["DUPLICATION"][1]+confmat_df.loc["OTHER-LINKS"][1]],
                               [confmat_df.loc["NON-LINKS"][0],
                                confmat_df.loc["NON-LINKS"][1]]]
                new_confmat_df = pd.DataFrame(new_confmat, index=['DUPLICATION', 'NON-LINKS'])
            else:
                new_confmat = [[confmat_df.loc["DUPLICATION"][0], 
                                confmat_df.loc["DUPLICATION"][1]],
                               [confmat_df.loc["OTHER-LINKS"][0]+confmat_df.loc["NON-LINKS"][0],
                                confmat_df.loc["OTHER-LINKS"][1]+confmat_df.loc["NON-LINKS"][1]]]
                new_confmat_df = pd.DataFrame(new_confmat, index=['DUPLICATION', 'NON-LINKS'])
            confmat_df = new_confmat_df

        tp, fp, tn, fn = get_tpfptnfn(confmat_df)
        
        d_pre = tp/(tp+fp)
        d_rec = tp/(tp+fn)
        d_f1 = 2*(d_pre*d_rec)/(d_pre+d_rec)

        avg_d_pre.append(d_pre)
        avg_d_rec.append(d_rec)
        avg_d_f1.append(d_f1)

        nl_pre = tn/(tn+fn)
        nl_rec = tn/(tn+fp)
        nl_f1 = 2*(nl_pre*nl_rec)/(nl_pre+nl_rec)

        avg_nl_pre.append(nl_pre)
        avg_nl_rec.append(nl_rec)
        avg_nl_f1.append(nl_f1)

        pre = (d_pre+nl_pre)/2
        rec = (d_rec+nl_rec)/2
        f1 = 2*(pre*rec)/(pre+rec)
        
        avg_pre.append(pre)
        avg_rec.append(rec)
        avg_f1.append(f1)
        
        acc = (tp+tn)/(tp+fp+fn+tn)

        avg_acc.append(acc)

    print("+++++++++++++++++")
    print("ACC: "+str(round(np.mean(avg_acc),2)))
    print("+++++++++++++++++")
    print("PRE: "+str(round(np.mean(avg_pre),2)))
    print("REC: "+str(round(np.mean(avg_rec),2)))
    print("F1: "+str(round(np.mean(avg_f1),2)))
    print("+++++++++++++++++")
    print("D PRE: "+str(round(np.mean(avg_d_pre),2)))
    print("D REC: "+str(round(np.mean(avg_d_rec),2)))
    print("D F1: "+str(round(np.mean(avg_d_f1),2)))
    print("+++++++++++++++++")
    print("NL PRE: "+str(round(np.mean(avg_nl_pre),2)))
    print("NL REC: "+str(round(np.mean(avg_nl_rec),2)))
    print("NL F1: "+str(round(np.mean(avg_nl_f1),2)))
    print("+++++++++++++++++")
    print("OL 0: "+str(round(np.mean(avg_ol_0),2)))
    print("OL 1: "+str(round(np.mean(avg_ol_1),2)))

    print("OL STD: "+str(round(np.std(avg_ol_0),2)))
    
    res_dict = {
            'ACC' : avg_acc,
            'Pre': avg_pre,
            'Rec': avg_rec,
            'F1': avg_f1,
            'D_Pre': avg_d_pre,
            'D_Rec': avg_d_rec,
            'D_F1': avg_d_f1,
            'NL_Pre': avg_nl_pre,
            'NL_Rec': avg_nl_rec,
            'NL_F1': avg_nl_f1,
            'OL_Corr': avg_ol_0,
          }

    res_data= pd.DataFrame(res_dict, index=[valid_projects])
    
    return res_data

In [10]:
get_results('R_LTvNL', True)

+++++++++++++++++
ACC: 0.65
+++++++++++++++++
PRE: 0.65
REC: 0.65
F1: 0.65
+++++++++++++++++
D PRE: 0.65
D REC: 0.64
D F1: 0.65
+++++++++++++++++
NL PRE: 0.65
NL REC: 0.66
NL F1: 0.65
+++++++++++++++++
OL 0: 0.56
OL 1: 0.44
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.650909,0.650941,0.650909,0.650925,0.653137,0.643636,0.648352,0.648746,0.658182,0.65343,0.556364


In [11]:
get_results('R_LTvNL', False)

+++++++++++++++++
ACC: 0.62
+++++++++++++++++
PRE: 0.61
REC: 0.63
F1: 0.62
+++++++++++++++++
D PRE: 0.45
D REC: 0.64
D F1: 0.53
+++++++++++++++++
NL PRE: 0.77
NL REC: 0.61
NL F1: 0.68
+++++++++++++++++
OL 0: 0.56
OL 1: 0.44
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.619394,0.611765,0.625455,0.618534,0.450382,0.643636,0.52994,0.773148,0.607273,0.680244,0.556364


In [12]:
get_results('R_LTvNLOL', True)

+++++++++++++++++
ACC: 0.61
+++++++++++++++++
PRE: 0.62
REC: 0.61
F1: 0.62
+++++++++++++++++
D PRE: 0.6
D REC: 0.68
D F1: 0.64
+++++++++++++++++
NL PRE: 0.63
NL REC: 0.55
NL F1: 0.59
+++++++++++++++++
OL 0: 0.58
OL 1: 0.42
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.614545,0.616324,0.614545,0.615433,0.601942,0.676364,0.636986,0.630705,0.552727,0.589147,0.581818


In [13]:
get_results('R_LTvNLOL', False)

+++++++++++++++++
ACC: 0.6
+++++++++++++++++
PRE: 0.61
REC: 0.62
F1: 0.62
+++++++++++++++++
D PRE: 0.44
D REC: 0.68
D F1: 0.53
+++++++++++++++++
NL PRE: 0.78
NL REC: 0.57
NL F1: 0.66
+++++++++++++++++
OL 0: 0.58
OL 1: 0.42
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.603636,0.608367,0.621818,0.615019,0.438679,0.676364,0.532189,0.778055,0.567273,0.656151,0.581818


In [14]:
get_results('R_LTOLvNL', True)

+++++++++++++++++
ACC: 0.6
+++++++++++++++++
PRE: 0.6
REC: 0.6
F1: 0.6
+++++++++++++++++
D PRE: 0.61
D REC: 0.53
D F1: 0.57
+++++++++++++++++
NL PRE: 0.59
NL REC: 0.67
NL F1: 0.62
+++++++++++++++++
OL 0: 0.43
OL 1: 0.57
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.598182,0.599992,0.598182,0.599086,0.613445,0.530909,0.569201,0.586538,0.665455,0.623509,0.425455


In [15]:
get_results('R_LTOLvNL', False)

+++++++++++++++++
ACC: 0.59
+++++++++++++++++
PRE: 0.6
REC: 0.61
F1: 0.6
+++++++++++++++++
D PRE: 0.77
D REC: 0.55
D F1: 0.64
+++++++++++++++++
NL PRE: 0.43
NL REC: 0.67
NL F1: 0.52
+++++++++++++++++
OL 0: 0.43
OL 1: 0.57
OL STD: 0.0


,ACC,Pre,Rec,F1,D_Pre,D_Rec,D_F1,NL_Pre,NL_Rec,NL_F1,OL_Corr
MariaDB,0.590303,0.597125,0.609091,0.603049,0.767677,0.552727,0.642706,0.426573,0.665455,0.519886,0.425455
